# Web scraping - eBay


# Part 1

a)	go to https://www.ebay.com and search for "lg phone" 

b)	what type of search request is eBay using, GET or POST?

    a.	eBay is using a GET request for the search 

c)	which URL variable represents the search term?

    a.	The URL variable “_nkw” represents the search term

d)	click on “Auction”. Which URL variable represents auction searches? 

    a.	The variable “LH_Auction” represents auction searches

e)	can you come up with a shorter URL that produces the same search result page? 

    a.	https://www.ebay.com/sch/?_nkw=lg%20phone&LH_Auction=1

f)	click on the next search result page and observe how the URL changes. What variable in the URL identifies the page number? 

    a.	The variable “_pgn” identifies the page number

g)	what is the feature common to each item in the search results page? I.e., what item do we need to select to obtain each item among the search results? 

    a.	The items are passed as parts of an unordered list <ul> "srp-results srp-list clearfix". Items in the list <li> are of the class “s-item s-item__pl-on-bottom”. They are differentiated using the “data-view” variable with an ID called “iid”.	

h)	identify the number of bids for each item in search results. What do they have in common? How does it look in the HTML source code? 

    a.	The number of bids is stored in a span of the same class across the items. It is as follows in the HTML source code: 

    “<span class="s-item__bids s-item__bidCount">4 bids</span>” 

    The actual bids inside the span changes across the items.


# Part 2

In [3]:
import requests
import time
from bs4 import BeautifulSoup

In [2]:
url_lgphone = "https://www.ebay.com/sch/?_nkw=lg%20phone"
header = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" ,
"authority": "www.tagesschau.de",
"method": "GET",
"path":"/",
"scheme":"https",
"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
"accept-encoding": "gzip, deflate, br",
"accept-language": "en-US,en;q=0.9,de;q=0.8",
"cache-control": "max-age=0",
"cookie": "atuserid=%7B%22name%22%3A%22atuserid%22%2C%22val%22%3A%2257ea5dd6-4c35-4982-942f-8a7f8b8c3a4b%22%2C%22options%22%3A%7B%22end%22%3A%222023-02-17T05%3A02%3A59.936Z%22%2C%22path%22%3A%22%2F%22%7D%7D; atidvisitor=%7B%22name%22%3A%22atidvisitor%22%2C%22val%22%3A%7B%22vrn%22%3A%22-595936-%22%7D%2C%22options%22%3A%7B%22path%22%3A%22%2F%22%2C%22session%22%3A15724800%2C%22end%22%3A15724800%7D%7D",
"sec-ch-ua-mobile": "?0",
"sec-ch-ua-platform": "Windows",
"sec-fetch-dest": "document",
"sec-fetch-mode": "navigate",
"sec-fetch-site": "none",
"sec-fetch-user": "?1",
"upgrade-insecure-requests": "1"}

In [3]:
##### Part a - Storing the HTML into a file

#Getting the page into a variable
lgphone_page = requests.get(url = url_lgphone, headers = header)

#Creating a new file
ebay_html = open("ebay_lg_phone.html", "w")
#The parameter "w" creates a file only if it does not exist already


#Adding the html content to the page using the write() function

ebay_html.write(lgphone_page.text)

print("The search results have been stored in a HTML file.")

The search results have been stored in a HTML file.


In [10]:
##### Part b - Storing the first 10 pages into 10 HTML files

#Creating the page number variable _pgn

pgn = list(range(1,11)) #Stores page numbers from 1-10
webpages_lg = list(range(1,11)) #To store the webpage output

for i in range(0, 10):
    #Appending the auction flag and page number.
    url_part2 = f"{url_lgphone}&LH_Auction=1&_pgn={i+1}" 
    lgphone_page_part2 = requests.get(url = url_part2, headers = header) #GET
    webpages_lg[i] = lgphone_page_part2 #Storing each page in a list.
    ebay_html = open(f"ebay_lg_phone{i+1}.html", "w") #Creating individual files.
    ebay_html.write(webpages_lg[i].text) #Writing to HTML files.
    print(f"Page {i+1} scraped.")
    time.sleep(15) #Adding sleep time to reduce frequency.

print("The search results have been stored in HTML files.")


Page 1 scraped.
Page 2 scraped.
Page 3 scraped.
Page 4 scraped.
Page 5 scraped.
Page 6 scraped.
Page 7 scraped.
Page 8 scraped.
Page 9 scraped.
Page 10 scraped.
The search results have been stored in HTML files.


In [12]:
##### Part c - Getting number of bids and item URL

#Creating a list to import the webpages back
webpages_lg_import = list(range(1,11))

for i in range(1,11):
    
    print(f"Products from Page {i}")
    count_items = 0
    #Each item in the above list stores an output from the html file
    webpages_lg_import[i-1] = open (f"ebay_lg_phone{i}.html", "r")
    
    #Parsing the html
    deals_parsed = BeautifulSoup(webpages_lg_import[i-1], 'lxml')

    #Finding all the unordered lists of class "srp-results srp-list clearfix"
    full_list = deals_parsed.find("ul",class_="srp-results srp-list clearfix")

    #Getting header, number of bids and link of each item in the unordered list
    #Each item is stored in a list - which is stored using the variable li in the following code

    for  li in full_list.find_all("li", class_ = "s-item s-item__pl-on-bottom s-item--watch-at-corner"):
        print(f"Product name : {li.find('h3').text}") #Getting the title of the product
        print(li.find('span', class_='s-item__bids s-item__bidCount').text) #Number of bids
        print(f"Link : {li.a.get('href')}") #Link to the product
        print(f"\n")
        count_items +=1
    
    #For the last item, the class is different. Hence we will use the below case
    for  li in full_list.find_all("li", 
                                  class_ = 
                                  "s-item s-item__pl-on-bottom s-item--watch-at-corner s-item__before-answer"):
        print(f"Product name : {li.find('h3').text}") #Getting the title of the product
        print(li.find('span', class_='s-item__bids s-item__bidCount').text) #Number of bids
        print(f"Link : {li.a.get('href')}") #Link to the product
        print(f"\n")
        count_items +=1
    
    print(f"----------{count_items} is the item count on this page---------------")
    
print("Products from all selected pages have been listed.")



Products from Page 1
Product name : LG V60 ThinQ 5G LMV600TM - 128GB - Classy Blue Unlocked - Please read
22 bids
Link : https://www.ebay.com/itm/115222274904?epid=17037601713&hash=item1ad3c86358%3Ag%3AGDoAAOSwRmVh8Haq&LH_Auction=1


Product name : New ListingLG G8x ThinQ - 128GB - Aurora Black (Unlocked) Smartphone
5 bids
Link : https://www.ebay.com/itm/234395990622?epid=11044531299&hash=item369316fa5e%3Ag%3Aq2QAAOSwVORh8zbH&LH_Auction=1


Product name : New ListingLG Stylo 6 LMQ730QM7 - 64GB - Holographic White (Factory Unlocked) - New
1 bid
Link : https://www.ebay.com/itm/284625951077?epid=24040441675&hash=item4245075965%3Ag%3ACBkAAOSwbO5h8T%7Ek&LH_Auction=1


Product name : New ListingLG LM600TM V60 THINQ 5G Dual Screen Smartphone 128GB T-Mobile 6.8" Blue
10 bids
Link : https://www.ebay.com/itm/234395374237?epid=21037603207&hash=item36930d929d%3Ag%3AuQcAAOSwr-1h8rMh&LH_Auction=1


Product name : LG V60 ThinQ 5G LMV600VM - 128GB - Classy Blue (Verizon) (Single SIM)
3 bids
Link : htt

Product name : LG V20 - VS995 CELL PHONE - USED APPROXIMATELY (1)  MONTH
30 bids
Link : https://www.ebay.com/itm/224805067202?epid=0&hash=item34576d19c2%3Ag%3AzCkAAOSwLslh00uR&LH_Auction=1


----------60 is the item count on this page---------------
Products from Page 2
Product name : LG G5 VS987 - 32GB - Silver (Verizon) Smartphone
0 bids
Link : https://www.ebay.com/itm/203802041614?epid=220957006&hash=item2f738c5d0e%3Ag%3AfioAAOSwylRh7Y9S&LH_Auction=1


Product name : LG G7 ThinQ G710ULM - 64GB - Matte Moroccan Blue (Google Fi) Used/functional
2 bids
Link : https://www.ebay.com/itm/255347024564?epid=14026794540&hash=item3b73de62b4%3Ag%3AzZAAAOSwV9Zh7NBC&LH_Auction=1


Product name : LG Velvet 5G LMG900UM1 - 128GB - Aurora Silver (AT&T) (Single SIM)
2 bids
Link : https://www.ebay.com/itm/363701458971?epid=4039854341&hash=item54ae4bd81b%3Ag%3Ao58AAOSw-jJh7bL5&LH_Auction=1


Product name : LG V20 - VS995 CELL PHONE - USED APPROXIMATELY (1)  MONTH
30 bids
Link : https://www.ebay.com/itm/

Product name : LG Stylo 5 LMQ720 - 32GB - Silvery Black (Metro) (Single SIM)
2 bids
Link : https://www.ebay.com/itm/115217986739?epid=15039525641&hash=item1ad386f4b3%3Ag%3ArUkAAOSwZxth7cLR&LH_Auction=1


----------60 is the item count on this page---------------
Products from Page 3
Product name : New ListingLG V40 ThinQ, 64GB, Aurora Black, Unlocked, Great Condition : GG566
1 bid
Link : https://www.ebay.com/itm/275141181302?epid=8024885378&hash=item400fb13f76%3Ag%3A4N0AAOSwFPFh7wLL&LH_Auction=1


Product name : LG Velvet 5G LMG900TM - 128GB - Aurora Gray (T-Mobile) (Single SIM)
11 bids
Link : https://www.ebay.com/itm/255346225724?epid=28041264968&hash=item3b73d2323c%3Ag%3A7%7EkAAOSwlQ1h61ne&LH_Auction=1


Product name : LG LM600TM V60 THINQ 5G Dual Screen Smartphone 128GB T-Mobile 6.8" Blue
49 bids
Link : https://www.ebay.com/itm/393899325153?epid=21037603207&hash=item5bb63ab6e1%3Ag%3AYR0AAOSwqh5h8Dxa&LH_Auction=1


Product name : New ListingLG Stylo 6 - 64GB  Boost Mobile White
1 bid

Product name : UNLOCKED LG V30 VS996 64GB 4G LTE Smart Phone / LYCA AT&T Metro T-Mobile Verizon
5 bids
Link : https://www.ebay.com/itm/334302640803?epid=15028524706&hash=item4dd5fd7aa3%3Ag%3Af5QAAOSw88FhWjfZ&LH_Auction=1


Product name : New ListingLG Stylo 2 VS835, 16GB, Verizon Locked, Gray, Good Condition : FF193
1 bid
Link : https://www.ebay.com/itm/275141088696?hash=item400fafd5b8%3Ag%3AYsIAAOSweyxhu5MU&LH_Auction=1


Product name : LG G8X ThinQ Dual Screen- 128GB - Aurora Black (Unlocked Sprint)
0 bids
Link : https://www.ebay.com/itm/125118819599?hash=item1d21a9ad0f%3Ag%3AeSoAAOSwoX9h3KUW&LH_Auction=1


Product name : LG-B450 Blue T-Mobile Flip Phone
9 bids
Link : https://www.ebay.com/itm/265512554430?epid=224829963&hash=item3dd1c80fbe%3Ag%3AUyoAAOSwwDZh4goZ&LH_Auction=1


Product name : LG STYLO 5, 32GB, BLACK, LOCKED TO CRICKET NETWORK 
0 bids
Link : https://www.ebay.com/itm/154807201173?hash=item240b3a6d95%3Ag%3AB7cAAOSwn9hh7G6V&LH_Auction=1


Product name : Cricket LG Fortune

Product name : New ListingLG G8 LM-G820UMB 128GB Platinum Gray Verizon Fully Functional Black Spot
0 bids
Link : https://www.ebay.com/itm/265521382326?epid=11031128194&hash=item3dd24ec3b6%3Ag%3AzCkAAOSw8Ndh8yV4&LH_Auction=1


Product name : New ListingLG V35 ThinQ (AT&T)  V350A 4G LTE 6.0" 16MP GSM Smartphone 
0 bids
Link : https://www.ebay.com/itm/203805872916?epid=14033714516&hash=item2f73c6d314%3Ag%3AstUAAOSwiith8we7&LH_Auction=1


Product name : LG K51 32GB Boost Mobile & Samsung Galaxy A20 Metro PCS For Parts Lot
15 bids
Link : https://www.ebay.com/itm/175121984858?epid=14033146055&hash=item28c6156d5a%3Ag%3AREQAAOSwn8dh60MW&LH_Auction=1


Product name : New ListingLG V20 , 64GB , Titan ,Sprint Locked, Sprint Locked, Glass Cracked : GG545
0 bids
Link : https://www.ebay.com/itm/275141137989?epid=232115107&hash=item400fb09645%3Ag%3A6QMAAOSwAIph7uef&LH_Auction=1


Product name : LG Optimus Elite Smart Cellphone white
0 bids
Link : https://www.ebay.com/itm/115217851193?hash=item1ad384e

Product name : LG G5 H850 - 32GB - Gold (Unlocked) Smartphone
0 bids
Link : https://www.ebay.com/itm/255350158093?epid=222037663&hash=item3b740e330d%3Ag%3ATlQAAOSw-zBh7xzK&LH_Auction=1


Product name : LG G7 Fit - 32GB - Black (Unlocked) LM-Q850QM (CDMA + GSM)
0 bids
Link : https://www.ebay.com/itm/325013810678?epid=7036211980&hash=item4bac552df6%3Ag%3A0ncAAOSwypJh0yxk&LH_Auction=1


Product name : LG Wing 5G F100TM 256GB 8GB Aurora Gray Dual Screen Swivel Phone T-Mobile 
0 bids
Link : https://www.ebay.com/itm/224808942348?epid=12049155199&hash=item3457a83b0c%3Ag%3AdWgAAOSwVWth8aIB&LH_Auction=1


Product name : New ListingLG K20 plus TP260 - 32GB - Black (T-Mobile) Smartphone New, No SIM card
0 bids
Link : https://www.ebay.com/itm/294771687228?epid=239081436&hash=item44a1c2ff3c%3Ag%3A3w8AAOSwDrVbJmsg&LH_Auction=1


Product name : LG Stylo 4 LG-Q710AL (Black 32GB)  AS IS NO RETURNS AO4033236
1 bid
Link : https://www.ebay.com/itm/334300156796?hash=item4dd5d7937c%3Ag%3Ah2cAAOSw3PBhJSed&LH

Product name : LG Risio 4 - 16gb Blue Smartphone - Cricket Wireless #4(56/20)
0 bids
Link : https://www.ebay.com/itm/275137560775?epid=21044153547&hash=item400f7a00c7%3Ag%3AqHMAAOSw-ERh79rD&LH_Auction=1


Product name : LG K40 LMX420 - 16GB - Gray (Unlocked) (Single SIM)
0 bids
Link : https://www.ebay.com/itm/294754620871?epid=3033828463&hash=item44a0be95c7%3Ag%3AO9gAAOSwaLRhz4Xb&LH_Auction=1


Product name : LG  32GB LS 990 0604  (Boost) Android Smartphone BdESN (B-125)for parts.it works
0 bids
Link : https://www.ebay.com/itm/194768720997?hash=item2d591ed465%3Ag%3AuzcAAOSw719ffAV1&LH_Auction=1


Product name : Flip Phone lg b470 
0 bids
Link : https://www.ebay.com/itm/154810754740?epid=1185944785&hash=item240b70a6b4%3Ag%3AVsMAAOSwUvVh5jNI&LH_Auction=1


Product name : LG Arena 2 - X320APM - AT&T - Grey - Smartphone - READ DESC!!!
0 bids
Link : https://www.ebay.com/itm/334302613781?hash=item4dd5fd1115%3Ag%3AiHMAAOSw1Nhh7elA&LH_Auction=1


Product name : LG Journey LTE LGL322DL - 16GB -